# Measurement Kata Workbook

**What is this workbook?**
A workbook is a collection of problems, accompanied by solutions to them. 
The explanations focus on the logical steps required to solve a problem; they illustrate the concepts that need to be applied to come up with a solution to the problem, explaining the mathematical steps required. 

Note that a workbook should not be the primary source of knowledge on the subject matter; it assumes that you've already read a tutorial or a textbook and that you are now seeking to improve your problem-solving skills. You should attempt solving the tasks of the respective kata first, and turn to the workbook only if stuck. While a textbook emphasizes knowledge acquisition, a workbook emphasizes skill acquisition.

This workbook describes the solutions to the problems offered in the [Measurement kata](./Measurements.ipynb). Since the tasks are offered as programming problems, the explanations also cover some elements of Q# that might be non-obvious for a first-time user.

**What you should know for this workbook**

You should be familiar with the following concepts before tackling the Measurements kata (and this workbook):

1.	Basic linear algebra
2.	The concept of qubit and multi-qubit systems
3.	Single-qubit and multi-qubit quantum gates and using them to manipulate the state of the system

To begin, first prepare this notebook for execution (if you skip the first step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells; if you skip the second step, you'll get "Invalid test name" error):

In [ ]:
%package Microsoft.Quantum.Katas::0.11.2003.3107

In [ ]:
%workspace reload

## Part I. Discriminating Orthogonal States

### Task 1.1. $|0\rangle$ or $|1\rangle$?

**Input:** A qubit which is guaranteed to be in either the $|0\rangle$ or the $|1\rangle$ state.

**Output:**  `true` if the qubit was in the $|1\rangle$ state, or `false` if it was in the $|0\rangle$ state. The state of the qubit at the end of the operation does not matter.

### Solution

The input qubit is guaranteed to be in either basis state $|0\rangle$ or $|1\rangle$, this means the when measuring the qubit in the Pauli $Z$ basis the input state will be known without a doubt.  
In Q# the `M()` function can be used to measure a single qubit in the Pauli $Z$ basis. This function will return `One` if the input qubit was in the $|1\rangle$ state and `Zero` if the input qubit was in the $|0\rangle$ state.

In [ ]:
%kata T101_IsQubitOne_Test 

operation IsQubitOne (q : Qubit) : Bool {
    return M(q) == One;
}

[Return to task 1.1 of the Measurements kata.](./Measurements.ipynb#Part-I.-Discriminating-Orthogonal-States)

### Task 1.2. Set the qubit to the $|0\rangle$ state.

**Input:** A qubit in an arbitrary state.

**Goal:**  Change the state of the qubit to $|0\rangle$.

### Solution

Regardless of the original qubit state, after we measure the qubit in the Pauli $Z$ basis, it will end up in the $|0\rangle$ or $|1\rangle$ state. This is an important property of quantum computing, when measuring a qubit a possible superposition will collapse.   
When the input state is known after measuring with `M()`, there are two possibilities:
1.	The qubit is in state $|0\rangle$, we don’t need to change anything
2.	The qubit is in state $|1\rangle$, we need to inverse the state of the qubit, this is done with the **X** gate (more information on the X gate in the [BasicGates kata](../BasicGates/BasicGates.ipynb)) 

In [ ]:
%kata T102_InitializeQubit_Test 

operation InitializeQubit (q : Qubit) : Unit {
    if (M(q) == One) {
        X(q);
    }
}

[Return to task 1.2 of the Measurements kata.](./Measurements.ipynb#Task-1.2.-Set-the-qubit-to-the-$|0\rangle$-state.)

### Task 1.3. $|+\rangle$ or $|-\rangle$?

**Input:** A qubit which is guaranteed to be in either the $|+\rangle$ or the $|-\rangle$ state. As a reminder, $|+\rangle = \frac{1}{\sqrt{2}} \big(|0\rangle + |1\rangle\big)$, $|-\rangle = \frac{1}{\sqrt{2}} \big(|0\rangle - |1\rangle\big)$.

**Output:** `true` if the qubit was in the $|+\rangle$ state, or `false` if it was in the $|-\rangle$ state. The state of the qubit at the end of the operation does not matter.

### Solution

The input state is a super position state, with equal amplitudes. This means if the sate is measured in the Pauli $Z$ basis like the previous task there would be a 50% - %50 change of measuring `One` or `Zero`.  
To be sure in which state the input qubit is we want to transform the input qubit into a state where there is no super position. 

Creating the input state is done like this: $H|0\rangle = |+\rangle$ and $H|1\rangle = |-\rangle$. This transformation can also be undone by Applying the **H** gate again on the input state this gives: $H|+\rangle = |0\rangle$  and $H|-\rangle = |1\rangle$.  
Once we have the $|0\rangle$ or $|1\rangle$ state we can use the same principle as the previous task to measure the state.

In [ ]:
%kata T103_IsQubitPlus_Test 

operation IsQubitPlus (q : Qubit) : Bool {
    H(q);
    return M(q) == Zero;
}

Another possible solution could be to measure in the Pauli $X$ basis, this means a transformation with the **H** gate is not needed.  
Measuring in another Pauli basis can be done with the [`Measure()`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.measure?view=qsharp-preview) function.

In [ ]:
%kata T103_IsQubitPlus_Test 

operation IsQubitPlus (q : Qubit) : Bool {
   return Measure([PauliX], [q]) == Zero;
}

[Return to task 1.3 of the Measurements kata.](./Measurements.ipynb#Task-1.3.-$|+\rangle$-or-$|-\rangle$?)

### Task 1.4. $|A\rangle$ or $|B\rangle$?

**Inputs:** 

1. Angle $\alpha$, in radians, represented as a `Double`.
2. A qubit which is guaranteed to be in either the $|A\rangle$ or the $|B\rangle$ state, where $|A\rangle = \cos \alpha |0\rangle + \sin \alpha |1\rangle$ and $|B\rangle = - \sin \alpha |0\rangle + \cos \alpha |1\rangle$.

**Output:** `true` if the qubit was in the $|A\rangle$ state, or `false` if it was in the $|B\rangle$ state. The state of the qubit at the end of the operation does not matter.

### Solution

We take a similar approach as the previous task.  
To create the input states $|A\rangle$ and $|B\rangle$ a [**Ry**](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.primitive.ry) gate with $\theta$ of $-2.0 \cdot \alpha$ was applied to the basis states $|0\rangle$ and $|1\rangle$.


$$R_y = \begin{bmatrix} cos(\frac{\theta}{2}) & - sin(\frac{\theta}{2}) \\ sin(\frac{\theta}{2}) & cos(\frac{\theta}{2}) \end{bmatrix}$$

We return the input state to the basis sates by applying **Ry** gate with $-2.0 \cdot \alpha$ as $\theta$ on the input qubit.

This gives two possibilities:
1. The qubit is measured as $|1\rangle$, the qubit is in state $|B\rangle$, we return `false`.
2. The qubit is measured as $|0\rangle$, the qubit is in state $|A\rangle$, we return `true`.

In [ ]:
%kata T104_IsQubitA_Test

operation IsQubitA (alpha : Double, q : Qubit) : Bool {
    Ry(-2.0 * alpha, q);
    return M(q) == Zero;
}

[Return to task 1.4 of the Measurements kata.](./Measurements.ipynb#Task-1.4.-$|A\rangle$-or-$|B\rangle$?)

### Task 1.5. $|00\rangle$ or $|11\rangle$?

**Input:** Two qubits (stored in an array of length 2) which are guaranteed to be in either the $|00\rangle$ or the $|11\rangle$ state.

**Output:** 0 if the qubits were in the $|00\rangle$ state, or 1 if they were in the $|11\rangle$ state. The state of the qubits at the end of the operation does not matter.

### Solution

Both of the qubits stored in the input array are in the same state, for $|00\rangle$ each individual qubit is in state $|0\rangle$, for $|11\rangle$ each individual qubit is in state $|1\rangle$. Therefore, if we measure one qubit we will know the state of the other qubit.  
In other words, if the first qubit measures as `One` we know that the qubits in the input array are in state $|11\rangle$ and vice versa.

In [ ]:
%kata T105_ZeroZeroOrOneOne_Test

operation ZeroZeroOrOneOne (qs : Qubit[]) : Int {
    return M(qs[0]) == One ? 1 | 0;
}

[Return to task 1.5 of the Measurements kata.](./Measurements.ipynb#Task-1.5.-$|00\rangle$-or-$|11\rangle$?)

### Task 1.6. Distinguish four basis states.

**Input:** Two qubits (stored in an array of length 2) which are guaranteed to be in one of the four basis states ($|00\rangle$, $|01\rangle$, $|10\rangle$, or $|11\rangle$).

**Output:**

* 0 if the qubits were in the $|00\rangle$ state,
* 1 if they were in the $|01\rangle$ state, 
* 2 if they were in the $|10\rangle$ state, 
* 3 if they were in the $|11\rangle$ state.

In this task and the subsequent ones the order of qubit states in task description matches the order of qubits in the array (i.e., $|10\rangle$ state corresponds to `qs[0]` in state $|1\rangle$ and `qs[1]` in state $|0\rangle$).

The state of the qubits at the end of the operation does not matter.

### Solution

In task 1.5 we wrote an operation to measure for the states $|00\rangle$ and $|11\rangle$, in this task we need to account for the mix-states $|01\rangle$ and $|10\rangle$. This time measuring the first qubit won't give use any information on the second qubit, so we need to measure both qubits.  
Firstly, we measure both qubits in the input array and store the result in `m1` and `m2`, with these results we can solve the task like this:  
- `m1` is $|0\rangle$ and `m2` is $|0\rangle$, we return 0*2+0 = 0
- `m1` is $|0\rangle$ and `m2` is $|1\rangle$, we return 0*2+1 = 1
- `m1` is $|1\rangle$ and `m2` is $|0\rangle$, we return 1*2+0 = 2
- `m1` is $|1\rangle$ and `m2` is $|1\rangle$, we return 1*2+1 = 3

In [ ]:
%kata T106_BasisStateMeasurement_Test

operation BasisStateMeasurement (qs : Qubit[]) : Int {
    // Measurement on the first qubit gives the higher bit of the answer, on the second - the lower
    let m1 = M(qs[0]) == Zero ? 0 | 1;
    let m2 = M(qs[1]) == Zero ? 0 | 1;
    return m1 * 2 + m2;
}

[Return to task 1.6 of the Measurements kata.](./Measurements.ipynb#Task-1.6.-Distinguish-four-basis-states.)

### Task 1.7. Distinguish two basis states given by bit strings

**Inputs:** 

1. $N$ qubits (stored in an array of length $N$) which are guaranteed to be in one of the two basis states described by the given bit strings.
2. Two bit strings represented as `Bool[]`s.

**Output:** 

* 0 if the qubits were in the basis state described by the first bit string,
* 1 if they were in the basis state described by the second bit string.

Bit values `false` and `true` correspond to $|0\rangle$ and $|1\rangle$ states. You are guaranteed that both bit strings have the same length as the qubit array, and that the bit strings differ in at least one bit.

You can use exactly one measurement. The state of the qubits at the end of the operation does not matter.

> Example:  for bit strings `[false, true, false]` and `[false, false, true]` return 0 corresponds to state $|010\rangle$, and return 1 corresponds to state $|001\rangle$.

### Solution

To solve this task we will use two steps. Like many other programing language in Q# you can write functions to make code more readable and more reusable. 

Step one is to find first bit strings that differs between `bit1` and `bit2`. For that we use the function `FindFirstDiff()` which loops through both `Bool[]`s and returns the index where the bit strings differ. 

In [ ]:
function FindFirstDiff_Reference (bits1 : Bool[], bits2 : Bool[]) : Int {
    for (i in 0 .. Length(bits1) - 1) {
        if (bits1[i] != bits2[i]) {
            return i;
        }
    }
    return -1;
}

The second step we will measure the qubit at the index which we have found in the previous step. If this qubit is in state $|0\rangle$ `res` takes the value false, if it is in state $|1\rangle$ it takes the value true.  
Lastly we check if the `true` or `false` value equals either the first or second bit string.

In [ ]:
%kata T107_TwoBitstringsMeasurement_Test

operation TwoBitstringsMeasurement (qs : Qubit[], bits1 : Bool[], bits2 : Bool[]) : Int {
    // find the first index at which the bit strings are different and measure it
    let firstDiff = FindFirstDiff_Reference(bits1, bits2);
    let res = M(qs[firstDiff]) == One;

    return res == bits1[firstDiff] ? 0 | 1;
}

[Return to task 1.7 of the Measurements kata.](./Measurements.ipynb#Task-1.7.-Distinguish-two-basis-states-given-by-bit-strings)

*To be continued...*